In [1]:
#importing the libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
#reading the data set
train_df = pd.read_csv("../data/train_folds.csv")

In [3]:
train_df.shape

(692, 15)

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692 entries, 0 to 691
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender                   692 non-null    float64
 1   Married                  692 non-null    float64
 2   Dependents               692 non-null    float64
 3   Education                692 non-null    float64
 4   Self_Employed            692 non-null    float64
 5   LoanAmount               692 non-null    float64
 6   Loan_Amount_Term         692 non-null    float64
 7   Credit_History           692 non-null    float64
 8   Total_Income             692 non-null    float64
 9   Semiurban                692 non-null    float64
 10  Urban                    692 non-null    float64
 11  Loan_Status              692 non-null    int64  
 12  total_income_by_loanAmt  692 non-null    float64
 13  Loan_Amount_Term_By_amt  692 non-null    float64
 14  kfold                    6

In [5]:
train_df.head()

,Gender,Married,Dependents,Education,Self_Employed,LoanAmount,Loan_Amount_Term,Credit_History,Total_Income,Semiurban,Urban,Loan_Status,total_income_by_loanAmt,Loan_Amount_Term_By_amt,kfold
0,1.0,1.0,0.000000,1.0,0.0,104.000000,360.000000,1.0,3819.000000,0.000000,1.0,1,1348.443140,0.288889,0
1,1.0,0.0,1.000000,1.0,0.0,117.000000,360.000000,1.0,6177.000000,0.000000,1.0,1,2787.298488,0.325000,0
2,1.0,1.0,2.419781,1.0,0.0,182.901096,334.813149,1.0,7142.630796,0.419781,0.0,0,1525.048970,0.546278,0
3,1.0,1.0,0.000000,1.0,0.0,143.701687,228.983805,0.0,7818.452295,0.000000,0.0,0,2960.178157,0.627563,0
4,1.0,1.0,0.000000,1.0,0.0,100.000000,360.000000,1.0,3539.000000,0.000000,1.0,1,1252.452100,0.277778,0


## Choosing the best model

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

In [8]:
def run_training(pred_df, fold, model, scale=False):
    
    #spliting the data into train and validation set
    train_df = pred_df[pred_df.kfold!=fold].reset_index(drop=True)
    valid_df = pred_df[pred_df.kfold==fold].reset_index(drop=True)

    X_train = train_df.drop(["Loan_Status"], axis=1).values
    X_valid = valid_df.drop(["Loan_Status"], axis=1).values
    
    if scale==True:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_valid = scaler.transform(X_valid)

    opt = model()
    opt.fit(X_train, train_df.Loan_Status.values)
    
    pred = opt.predict(X_valid)
    
    auc = roc_auc_score(valid_df.Loan_Status.values, pred)
    acc = accuracy_score(valid_df.Loan_Status.values, pred)

    return auc,acc

In [9]:
def train(df, model, scale=False):
    aucs = []
    accs = []
    for i in range(5):
        auc, acc = run_training(df, i, model, scale)
        aucs.append(auc)
        accs.append(acc)
    return aucs, accs

In [10]:
lr = train(train_df, LogisticRegression, scale=True)
svc = train(train_df, SVC, scale=True)
rf = train(train_df, RandomForestClassifier)
ada = train(train_df, AdaBoostClassifier, scale=True)
gb = train(train_df, GradientBoostingClassifier, scale=True)
xgb = train(train_df, XGBClassifier, scale=True)
cat = train(train_df, CatBoostClassifier, scale=True)

C:\Users\MOHAN\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:48:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\MOHAN\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:48:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\MOHAN\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\MOHAN\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

160:	learn: 0.4132746	total: 710ms	remaining: 3.7s
161:	learn: 0.4126982	total: 713ms	remaining: 3.69s
162:	learn: 0.4118354	total: 715ms	remaining: 3.67s
163:	learn: 0.4111439	total: 717ms	remaining: 3.65s
164:	learn: 0.4102168	total: 720ms	remaining: 3.64s
165:	learn: 0.4093594	total: 722ms	remaining: 3.63s
166:	learn: 0.4088025	total: 724ms	remaining: 3.61s
167:	learn: 0.4081183	total: 727ms	remaining: 3.6s
168:	learn: 0.4073424	total: 730ms	remaining: 3.59s
169:	learn: 0.4064468	total: 733ms	remaining: 3.58s
170:	learn: 0.4056248	total: 735ms	remaining: 3.56s
171:	learn: 0.4052749	total: 738ms	remaining: 3.55s
172:	learn: 0.4047409	total: 740ms	remaining: 3.54s
173:	learn: 0.4040939	total: 743ms	remaining: 3.53s
174:	learn: 0.4034988	total: 745ms	remaining: 3.51s
175:	learn: 0.4029045	total: 748ms	remaining: 3.5s
176:	learn: 0.4021659	total: 750ms	remaining: 3.49s
177:	learn: 0.4015619	total: 753ms	remaining: 3.48s
178:	learn: 0.4010375	total: 755ms	remaining: 3.46s
179:	learn: 0.4

346:	learn: 0.3239840	total: 1.4s	remaining: 2.63s
347:	learn: 0.3236517	total: 1.4s	remaining: 2.62s
348:	learn: 0.3231569	total: 1.41s	remaining: 2.62s
349:	learn: 0.3229021	total: 1.41s	remaining: 2.62s
350:	learn: 0.3225127	total: 1.41s	remaining: 2.61s
351:	learn: 0.3222175	total: 1.42s	remaining: 2.61s
352:	learn: 0.3217956	total: 1.42s	remaining: 2.61s
353:	learn: 0.3213999	total: 1.43s	remaining: 2.6s
354:	learn: 0.3211580	total: 1.43s	remaining: 2.6s
355:	learn: 0.3207746	total: 1.43s	remaining: 2.6s
356:	learn: 0.3205764	total: 1.44s	remaining: 2.59s
357:	learn: 0.3201063	total: 1.44s	remaining: 2.59s
358:	learn: 0.3197647	total: 1.45s	remaining: 2.58s
359:	learn: 0.3192723	total: 1.45s	remaining: 2.58s
360:	learn: 0.3187828	total: 1.46s	remaining: 2.58s
361:	learn: 0.3181858	total: 1.46s	remaining: 2.57s
362:	learn: 0.3177356	total: 1.46s	remaining: 2.57s
363:	learn: 0.3173116	total: 1.47s	remaining: 2.56s
364:	learn: 0.3167953	total: 1.47s	remaining: 2.56s
365:	learn: 0.316

519:	learn: 0.2719117	total: 2.09s	remaining: 1.93s
520:	learn: 0.2716632	total: 2.09s	remaining: 1.92s
521:	learn: 0.2714303	total: 2.09s	remaining: 1.92s
522:	learn: 0.2712023	total: 2.1s	remaining: 1.91s
523:	learn: 0.2709935	total: 2.1s	remaining: 1.91s
524:	learn: 0.2706739	total: 2.1s	remaining: 1.91s
525:	learn: 0.2704302	total: 2.11s	remaining: 1.9s
526:	learn: 0.2701761	total: 2.11s	remaining: 1.9s
527:	learn: 0.2699035	total: 2.12s	remaining: 1.89s
528:	learn: 0.2695932	total: 2.12s	remaining: 1.89s
529:	learn: 0.2692737	total: 2.12s	remaining: 1.88s
530:	learn: 0.2689863	total: 2.13s	remaining: 1.88s
531:	learn: 0.2687453	total: 2.13s	remaining: 1.87s
532:	learn: 0.2685315	total: 2.13s	remaining: 1.87s
533:	learn: 0.2683215	total: 2.14s	remaining: 1.86s
534:	learn: 0.2681286	total: 2.14s	remaining: 1.86s
535:	learn: 0.2679241	total: 2.15s	remaining: 1.86s
536:	learn: 0.2675225	total: 2.15s	remaining: 1.85s
537:	learn: 0.2672966	total: 2.16s	remaining: 1.85s
538:	learn: 0.267

696:	learn: 0.2319013	total: 2.92s	remaining: 1.27s
697:	learn: 0.2317614	total: 2.93s	remaining: 1.27s
698:	learn: 0.2315935	total: 2.93s	remaining: 1.26s
699:	learn: 0.2313530	total: 2.94s	remaining: 1.26s
700:	learn: 0.2311657	total: 2.94s	remaining: 1.25s
701:	learn: 0.2309841	total: 2.95s	remaining: 1.25s
702:	learn: 0.2307967	total: 2.95s	remaining: 1.25s
703:	learn: 0.2306946	total: 2.96s	remaining: 1.24s
704:	learn: 0.2304431	total: 2.96s	remaining: 1.24s
705:	learn: 0.2301561	total: 2.97s	remaining: 1.24s
706:	learn: 0.2298133	total: 2.98s	remaining: 1.23s
707:	learn: 0.2295570	total: 3s	remaining: 1.24s
708:	learn: 0.2294115	total: 3.01s	remaining: 1.24s
709:	learn: 0.2292360	total: 3.02s	remaining: 1.23s
710:	learn: 0.2290978	total: 3.03s	remaining: 1.23s
711:	learn: 0.2288514	total: 3.03s	remaining: 1.23s
712:	learn: 0.2286772	total: 3.04s	remaining: 1.22s
713:	learn: 0.2285300	total: 3.04s	remaining: 1.22s
714:	learn: 0.2282125	total: 3.05s	remaining: 1.22s
715:	learn: 0.2

901:	learn: 0.1924693	total: 3.62s	remaining: 393ms
902:	learn: 0.1923661	total: 3.62s	remaining: 389ms
903:	learn: 0.1921600	total: 3.62s	remaining: 385ms
904:	learn: 0.1919864	total: 3.63s	remaining: 381ms
905:	learn: 0.1918092	total: 3.63s	remaining: 376ms
906:	learn: 0.1916328	total: 3.63s	remaining: 372ms
907:	learn: 0.1915025	total: 3.63s	remaining: 368ms
908:	learn: 0.1913242	total: 3.63s	remaining: 364ms
909:	learn: 0.1911852	total: 3.64s	remaining: 360ms
910:	learn: 0.1909738	total: 3.64s	remaining: 356ms
911:	learn: 0.1908801	total: 3.64s	remaining: 352ms
912:	learn: 0.1907016	total: 3.65s	remaining: 348ms
913:	learn: 0.1905403	total: 3.65s	remaining: 343ms
914:	learn: 0.1903594	total: 3.65s	remaining: 339ms
915:	learn: 0.1902507	total: 3.65s	remaining: 335ms
916:	learn: 0.1901437	total: 3.66s	remaining: 331ms
917:	learn: 0.1899033	total: 3.66s	remaining: 327ms
918:	learn: 0.1897983	total: 3.66s	remaining: 323ms
919:	learn: 0.1896702	total: 3.66s	remaining: 319ms
920:	learn: 

In [11]:
print(f"Logistic Regression score: Auc = {np.mean(lr[0])}, Accuracy = {np.mean(lr[1])}")
print(f"Support Vector score: Auc = {np.mean(svc[0])}, Accuracy = {np.mean(svc[1])}")
print(f"Random Forest score: Auc = {np.mean(rf[0])}, Accuracy = {np.mean(rf[1])}")
print(f"Adaboost Score: Auc = {np.mean(ada[0])}, Accuracy = {np.mean(ada[1])}")
print(f"Gradient boosting score: Auc = {np.mean(gb[0])}, Accuracy = {np.mean(gb[1])}")
print(f"XGB score: Auc = {np.mean(xgb[0])}, Accuracy = {np.mean(xgb[1])}")
print(f"catboost score: Auc = {np.mean(cat[0])}, Accuracy = {np.mean(cat[1])}")

Logistic Regression score: Auc = 0.6820496894409939, Accuracy = 0.6822020644354082
Support Vector score: Auc = 0.7557142857142857, Accuracy = 0.7558752997601917
Random Forest score: Auc = 0.8671014492753623, Accuracy = 0.8671775622979876
Adaboost Score: Auc = 0.8366459627329192, Accuracy = 0.8367427796892921
Gradient boosting score: Auc = 0.8497308488612838, Accuracy = 0.8497549786257951
XGB score: Auc = 0.8381573498964803, Accuracy = 0.8382233343759775
catboost score: Auc = 0.8497308488612838, Accuracy = 0.849796684391617


In [12]:
model_scores = np.array([lr, svc, rf, ada, gb, xgb, cat])

In [13]:
performance = pd.DataFrame({
    "Model": ["Logistic Regression", "SVC", "Random Forest", "AdaBoost", "Gradient Boost","XGBoost", "CatBoost"],
    "Accuracy Score": model_scores[:,1,:].mean(axis=1), 
    "Roc Auc Score": model_scores[:,0,:].mean(axis=1)})

In [14]:
performance.sort_values(by=["Accuracy Score", "Roc Auc Score"], ascending=False, ignore_index=True)

,Model,Accuracy Score,Roc Auc Score
0,Random Forest,0.867178,0.867101
1,CatBoost,0.849797,0.849731
2,Gradient Boost,0.849755,0.849731
3,XGBoost,0.838223,0.838157
4,AdaBoost,0.836743,0.836646
5,SVC,0.755875,0.755714
6,Logistic Regression,0.682202,0.682050


#### Random Forest and Cat Boost have given the best results